In [1]:
from Acesso import Login
from Query import Query
import pandas as pd

s=Login()

sql=Query(s.usuario,s.senha,s.database,s.server)

pd.set_option('display.max_columns',None)


In [2]:
querys={
    
    'Dados':
    
    """
    
    SELECT cli.CNPJ,cli.Endereço,cli.Município,cli.UF,
    Produto,
    CONVERT(DECIMAL(15,2),COALESCE([Total Geral]/NULLIF(Qtde,0),0)) AS [Valor VDA],Qtde,
    CONVERT(INT,Qtde/NULLIF([Fator CX],0)) AS [Caixas],
    CHOOSE(Mês,'JANEIRO','FEVEREIRO','MARÇO','ABRIL','MAIO','JUNHO','JULHO','AGOSTO','SETEMBRO','OUTUBRO','NOVEMBRO','DEZEMBRO') AS 'Mês',
    [Ano]
    FROM (

        SELECT ped.[ID Cliente],ped.SKU,prod.Produto,prod.[Fator CX],
        SUM(ped.Qtde) AS [Qtde],sum([Total Geral]) AS [Total Geral],
        MONTH([Data de Faturamento]) AS [Mês],YEAR([Data de Faturamento]) AS [Ano]
        FROM netfeira.vw_targetestatico ped
        INNER JOIN netfeira.vw_produto prod ON ped.SKU=prod.SKU AND prod.Fabricante='BIMBO'
        WHERE ped.[Data de Faturamento] BETWEEN '2022-01-01' AND '2022-11-30' AND [Tipo de Operação]<>'OUTROS'
        GROUP BY ped.[ID Cliente],ped.SKU,prod.Produto,prod.[Fator CX],MONTH([Data de Faturamento]),
        YEAR([Data de Faturamento])

    )ev
    INNER JOIN netfeira.vw_cliente cli ON ev.[ID Cliente]=cli.[ID Cliente]
    
    """
    
}

In [3]:
df=sql.CriarTabela(kwargs=querys)

In [4]:
df['Dados'].to_excel('Consolidado.xlsx',index=False,encoding='UTF-8')